In [ ]:
from main import environment
import random,sys,gc,warnings
from IPython.display import clear_output
import gymnasium as gym 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from torch.utils.tensorboard import SummaryWriter

from tqdm import tqdm

In [ ]:
torch.autograd.set_detect_anomaly(True)
warnings.filterwarnings("ignore")

In [ ]:
env = gym.make("sudoku")

In [ ]:
# hypers
batchSize = 200
lr = 1e-4

In [ ]:
class mask: # altering softmax output so x and y = {0,8} and value = {1,9}
  def __init__(self):
    self.newValue = -float("inf")

  def apply(self,tensor : torch.FloatTensor):
    self.mask = torch.zeros_like(tensor,dtype=torch.bool)
    self.mask[0,-1] = True
    self.mask[1,-1] = True
    self.mask[-1,0] = True
    tensor = tensor.masked_fill(mask=self.mask,value=self.newValue)
    return tensor

class network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.LazyConv2d(1,(1,1))
    self.conv2 = nn.LazyConv2d(1,(1,1))
    self.conv3 = nn.LazyConv2d(1,(1,1))

    self.linear1 = nn.LazyLinear(9)
    self.linear2 = nn.LazyLinear(9)
    self.linear3 = nn.LazyLinear(9)

    self.policy_mask = mask()
    self.policy_head = nn.LazyLinear(30)
    self.value_head = nn.LazyLinear(1)
    
    self.optimizer = torch.optim.Adam(self.parameters(),lr=lr)
    
  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = self.conv2(x)
    x = F.relu(self.conv3(x))
    x = torch.flatten(x,start_dim=1)
    x = self.linear1(x)
    x = F.relu(self.linear2(x))
    x = self.linear3(x)
    distibution = F.relu(self.policy_head(x)).reshape(3,10)
    distibution = self.policy_mask.apply(distibution)
    value = self.value_head(x)
    return F.softmax(distibution,-1),value

d = network()
d.forward(torch.rand((1,9,9),dtype=torch.float))
clear_output()

In [ ]:
class collector:
    def __init__(self,totalFrame,batchSize):
        assert totalFrame % batchSize == 0 , f"TotalFrame / batchSize should yield 0"
        assert totalFrame < len(modifiableCells)*3 ,f"The memory lenght should be less than an episodes"
        self.totalFrame = totalFrame
        self.env = Env()
        self.network = network()
        self.pointer = 0
        self.data = []
        self.gamma = 0.99
        self.lambdaa = 0.99
     
    def rollout(self):
        self.clear() 
        while not len(self.data) == self.totalFrame : 
            if len(self.env.modifiableCells) < 5 :
                self.env.reset()
            softmax_dist,_value = self.network.forward(self.env.state.unsqueeze(0).clone())
            dist = Categorical(softmax_dist)
            sample = dist.sample()
            _log_prob = dist.log_prob(sample)
            action = sample.tolist()
            assert len(action) == 3 , f" action is {action}, problem during rollout"
            x,y,_ = action
            if (x,y) in self.env.modifiableCells:
                _state,_reward,_done,_action,_conflicts = self.env.step(action)
                self.data.append([_state,_reward,_action,_done,_conflicts,_log_prob,_value])
        
        # compute advantage
        _,_rewards,_,_,_,_,_value = zip(*self.data)
        re = torch.stack(_rewards,dim=-1).squeeze()
        val = torch.cat((torch.stack(_value,dim=-1).squeeze(),torch.tensor([0])),dim=-1)
        advantage = torch.zeros_like(re)
        gae = 0
        for n in reversed(range(len(re))):
            td = re[n] + self.gamma * val[n+1] - val[n]
            gae = td + self.gamma * self.lambdaa * gae
            advantage[n] = gae
    
        for data,advaaa in zip(self.data,torch.flip(advantage,dims=[-1])):
            data.append(advaaa)
        random.shuffle(self.data)  
     
    def sample(self,minibatches):
        output = self.data[self.pointer : self.pointer + minibatches]
        self.pointer += minibatches
        states,rewards,actions,dones,conflicts,logs,values,advantages = zip(*output)
        return states,rewards,dones,actions,logs,values,advantages,conflicts
    
    def clear(self):
        self.pointer = 0 
        self.data = []
        gc.collect()


t = collector(4,1)
t.rollout()

In [ ]:
class sudoku:
    def __init__(self):
        self.network = network()
        self.env = Env()
        self.totalFrame = 10#20
        self.memory = collector(self.totalFrame,self.batchSize )
        self.writter = SummaryWriter("./")

    def save(self):
        torch.save(self.network.state_dict(),"./policy.pth")

    def learn(self):
        for i in tqdm(range(self.epochs),total=self.epochs):
            self.memory.clear() 
            self.memory.rollout()

            for p in range(1): # minibatches
                states,rewards,dones,actions,oldProbs,values,advantages,conf = self.memory.sample(5)
                
                for y in range(5): # epochs optim
                    con = torch.mean(torch.stack(conf)).floor()
                    r = torch.mean(torch.stack(rewards)) 
                    ad = torch.stack(advantages,dim=-1).squeeze()
                    val = torch.stack(values,dim=-1).squeeze()
                    
                    vtarget = ad + val
                    critic_loss = torch.mean(torch.pow((val-vtarget),2))
                    # 
                    sta = torch.stack(states,dim=0)
                    act = torch.stack(actions,dim=0)
                    probs,_ = self.network.forward(sta)
                    dist = Categorical(probs)
                    np = dist.log_prob(act)
                    ratio = torch.exp(np) / torch.exp(torch.stack(oldProbs))
                    actorLosslist = []
                    # policy loss
                    for i in range(len(advantages)):
                        ratioAdvantage = ratio*val[i]
                        clippedRatio = (torch.clamp(ratio,(1-self.epsilon),(1+self.epsilon))*val[i])
                        policyLoss = torch.min(ratioAdvantage,clippedRatio)
                        actorLosslist.append(policyLoss)
                    actorLoss = -torch.mean(torch.stack(actorLosslist))
                
                    totalLoss = actorLoss + self.c1*critic_loss # actorLoss + (weight critic loss  * critic loss
                    self.network.optimizer.zero_grad()
                    totalLoss.backward(retain_graph=True)
                    self.network.optimizer.step()

            self.writter.add_scalar("main/conflicts",con)
            self.writter.add_scalar("main/Loss",totalLoss)
            self.writter.add_scalar("main/reward",r)

        self.save()
        

In [ ]:
#%load_ext tensorboard
#%tensorboard --logdir "./"

$$
{\small

\begin{align}

&\delta_t = r_t + \gamma V(s_{t+1}) - V(s_t) \\
& \hat{A_t} = \delta_t + (\gamma \lambda)\delta_{t+1}\\
&GAE : \hat{A_t} = \delta_t + (\gamma \lambda)\hat{A}_{t+1} \\[3em]

&L_\text{critic} = \frac{1}{N} \sum_t \left( V_t - V_t^\text{target} \right)^2 \\
& V_t^\text{target} = \hat{A_t} + V(s_t) \\[3em]

&L^{CPI} = \mathbb{\hat{E_{t}}}
\begin{bmatrix}
\frac{\pi_{\theta}(a_t | s_t)}{\pi_{\theta old}(a_t | s_t)}\hat{A_t}  \\
\end{bmatrix} \\

&\hspace{2em} = \mathbb{\hat{E_t}}
\begin{bmatrix}
r_t(\theta)\hat{A_t}  \\
\end{bmatrix} \\[1em]

&L^{CLIP} = \mathbb{\hat{E_t}}
\begin{bmatrix}
min(r_t(\theta) \hat{A_t},clip(r_t(\theta),1-\epsilon,1+\epsilon) \hat{A_t})
\end{bmatrix}\\[3em]

&Total Loss : L_t(\theta) = \mathbb{\hat{E_t}}
\begin{bmatrix}
L_t^{CLIP}{\theta} - c_1L_t^{critic}(\theta)
\end{bmatrix}\\

\end{align}
}
$$
